# LIBRARIES

In [22]:
library(httr)
library(jsonlite)
library(tidyverse)
library(dplyr)
library(tidyr)
library(testthat)

In [33]:
library(testthat)

# API KEY

In [2]:
api_key <- Sys.getenv("RAPID-API")
print(api_key)

[1] "c63e714db1msh82da74f7cae7ccbp1aeb67jsnf2684826358a"


# HELPER FUNCTIONS

In [3]:
makeApiRequest <- function(url, queryString, api_key) {
  
  # Make the GET request to the specified URL with given query parameters and headers
  response <- VERB("GET", url, query = queryString,
                         add_headers('X-RapidAPI-Key' = api_key,
                                           'X-RapidAPI-Host' = 'api-football-v1.p.rapidapi.com'),
                         content_type("application/octet-stream"))
  
  # Return the response object for further processing
  json_string <- content(response, "text")
  
  # Convert the JSON string to a dataframe
  df <- fromJSON(json_string, flatten = TRUE)
  df_data <- as.data.frame(df$response)

  return(df_data)
}


In [4]:
# Helper Funtion to print goal_scorer
goal_scorer_dashboard <- function(df) {
  # Sort the data frame by the number of goals in descending order
  top_scorers_sorted <- df %>% arrange(desc(goals.total))

  # Select the top 10 scorers
  top_10_scorers <- head(top_scorers_sorted, 10)

  
  league_name <- top_10_scorers$league.name[1]
  league_country <- top_10_scorers$league.country[1]

  # General League information
  cat("|------------------------|\n")
  cat("League Name:", league_name, "\n")
  cat("League Country:", league_country, "\n")
  cat("|------------------------|\n")

  # Print Top 10 goal Scorers
  for (i in 1:nrow(top_10_scorers)) {
    scorer <- top_10_scorers[i, ]
    cat("Team name:", scorer$team.name, "\n")
    cat("Player:", scorer$player.firstname, scorer$player.lastname, "\n")
    cat("Goals Scored:", scorer$goals.total, "\n")
    cat("|----------------------|\n")
  }
}

# TOP SCORER, TOP ASSISTS

In [5]:
getTopScorers <- function(league, season, api_key) {
  
  # Specific url to request
  url <- "https://api-football-v1.p.rapidapi.com/v3/players/topscorers"

  # Hyperparameters
  queryString <- list(
    league = league,
    season = season
  )

  # Call ApiRequest helper function to obtain dataframe
  df_data <- makeApiRequest(url, queryString, api_key)

  # Unnest Statistics column containing goal scorer data
  df_unnested <- df_data %>% unnest(statistics)
  
  # Select relevant columns from dataframe
  df_unnested <- select(df_unnested, player.firstname, player.lastname, player.nationality, games.position, team.name, 
                        league.name, games.appearences, goals.total, goals.assists, shots.total, shots.on)
  
  # Return the unnested data frame
  return(df_unnested)
}


In [6]:
top_scorers <- getTopScorers(39, 2020, api_key)

No encoding supplied: defaulting to UTF-8.



In [7]:
head(top_scorers)

player.firstname,player.lastname,player.nationality,games.position,team.name,league.name,games.appearences,goals.total,goals.assists,shots.total,shots.on
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>
Harry Edward,Kane,England,Attacker,Tottenham,Premier League,35,23,14,110,53
Mohamed,Salah Hamed Mahrous Ghaly,Egypt,Attacker,Liverpool,Premier League,37,22,5,90,52
Bruno Miguel,Borges Fernandes,Portugal,Midfielder,Manchester United,Premier League,37,18,12,84,51
Heung-Min,Son,Korea Republic,Attacker,Tottenham,Premier League,37,17,10,53,36
Patrick James,Bamford,England,Attacker,Leeds,Premier League,38,17,7,90,47
Dominic Nathaniel,Calvert-Lewin,England,Attacker,Everton,Premier League,33,16,NA,72,46


In [8]:
getTopAssists <- function(league, season, api_key) {
  
  # Specific url to request
  url <- "https://api-football-v1.p.rapidapi.com/v3/players/topassists"

  # Hyperparameters
  queryString <- list(
    league = league,
    season = season
  )

  # Call ApiRequest helper function to obtain dataframe
  df_data <- makeApiRequest(url, queryString, api_key)

  # Unnest Statistics column containing goal scorer data
  df_unnested <- df_data %>% unnest(statistics)
  
  # Select relevant columns from dataframe
  df_unnested <- select(df_unnested, player.firstname, player.lastname, player.nationality, games.position, team.name, league.name,
                        games.appearences, goals.assists, goals.total, passes.total, passes.key, passes.accuracy)
  
  # Return the unnested data frame
  return(df_unnested)
}


In [9]:
top_assists <- getTopAssists(39, 2020, api_key)

No encoding supplied: defaulting to UTF-8.



In [10]:
head(top_assists)

player.firstname,player.lastname,player.nationality,games.position,team.name,league.name,games.appearences,goals.assists,goals.total,passes.total,passes.key,passes.accuracy
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>
Harry Edward,Kane,England,Attacker,Tottenham,Premier League,35,14,23,909,51,18
Bruno Miguel,Borges Fernandes,Portugal,Midfielder,Manchester United,Premier League,37,12,18,2109,97,44
Kevin,De Bruyne,Belgium,Midfielder,Manchester City,Premier League,25,12,6,1263,81,41
Heung-Min,Son,Korea Republic,Attacker,Tottenham,Premier League,37,10,17,1056,76,23
Jack Peter,Grealish,England,Midfielder,Aston Villa,Premier League,26,10,6,996,82,31
Jamie Richard,Vardy,England,Attacker,Leicester,Premier League,34,9,15,434,28,8


# FIXTURES

In [11]:
getFootballFixtures <- function(league, season, api_key) {
  
  # Specific url to request
  url <- "https://api-football-v1.p.rapidapi.com/v3/fixtures"
  
  # Hyperparameters
  queryString <- list(
    league = league,
    season = season
  )
  
  # Call ApiRequest helper function to obtain dataframe
  df_data <- makeApiRequest(url, queryString, api_key)

  # Select relevant columns from dataframe
  df_data <- select(df_data, league.name, fixture.venue.name, fixture.date, 
                           fixture.status.long, teams.home.name, teams.away.name, 
                           score.fulltime.home, score.fulltime.away)
  
  return(df_data)
}


In [12]:
fixtures <- getFootballFixtures(league = 39, season = 2023, api_key)

No encoding supplied: defaulting to UTF-8.



In [13]:
head(fixtures)

,league.name,fixture.venue.name,fixture.date,fixture.status.long,teams.home.name,teams.away.name,score.fulltime.home,score.fulltime.away
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>
1,Premier League,Turf Moor,2023-08-11T19:00:00+00:00,Match Finished,Burnley,Manchester City,0,3
2,Premier League,Emirates Stadium,2023-08-12T11:30:00+00:00,Match Finished,Arsenal,Nottingham Forest,2,1
3,Premier League,Vitality Stadium,2023-08-12T14:00:00+00:00,Match Finished,Bournemouth,West Ham,1,1
4,Premier League,The American Express Community Stadium,2023-08-12T14:00:00+00:00,Match Finished,Brighton,Luton,4,1
5,Premier League,Goodison Park,2023-08-12T14:00:00+00:00,Match Finished,Everton,Fulham,0,1
6,Premier League,Bramall Lane,2023-08-12T14:00:00+00:00,Match Finished,Sheffield Utd,Crystal Palace,0,1


# LEAGUE STANDINGS


In [14]:
getFootballStandings <- function(league, season, api_key) {
  
  # Specific url to request
  url <- "https://api-football-v1.p.rapidapi.com/v3/standings"
  
  # Hyperparameters
  queryString <- list(
    league = league,
    season = season
  )
  
  # Call ApiRequest helper function to obtain dataframe
  df_data <- makeApiRequest(url, queryString, api_key)

  # Select relevant columns from dataframe
  df_data <- df_data$league.standings

  # Unlisting twice to obtain df
  inner_list <- df_data[[1]]
  standings_df <- inner_list[[1]]
  
  # Select relevant columns from dataframe
  standings_df <- select(standings_df, rank, team.name, group, description, form, points, all.played,
                         all.win, all.draw, all.lose, all.goals.for, all.goals.against, goalsDiff)

  return(standings_df)
}


In [15]:
standings <- getFootballStandings(39, 2020, api_key)

No encoding supplied: defaulting to UTF-8.



In [16]:
head(standings)

,rank,team.name,group,description,form,points,all.played,all.win,all.draw,all.lose,all.goals.for,all.goals.against,goalsDiff
,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,1,Manchester City,Premier League,Promotion - Champions League (Group Stage),WLWLW,86,38,27,5,6,83,32,51
2,2,Manchester United,Premier League,Promotion - Champions League (Group Stage),WDLLW,74,38,21,11,6,73,44,29
3,3,Liverpool,Premier League,Promotion - Champions League (Group Stage),WWWWW,69,38,20,9,9,68,42,26
4,4,Chelsea,Premier League,Promotion - Champions League (Group Stage),LWLWW,67,38,19,10,9,58,36,22
5,5,Leicester,Premier League,Promotion - Europa League (Group Stage),LLWLD,66,38,20,6,12,68,50,18
6,6,West Ham,Premier League,Promotion - Europa League (Group Stage),WWDLW,65,38,19,8,11,62,47,15


# TRANSFERS

In [18]:
getTeamTransfers <- function(team, api_key) {
    
    url <- "https://api-football-v1.p.rapidapi.com/v3/transfers"

    queryString <- list(team = "33")

    df_data <- makeApiRequest(url, queryString, api_key)
    
    expanded_df <- unnest(df_data, cols = transfers)

    expanded_df <- expanded_df %>%
    mutate(date = as.Date(date))

    result_df <- expanded_df %>%
    select(-1) %>%
    arrange(desc(date))

    return(result_df)
}

In [19]:
transfers <- getTeamTransfers(team = '33', api_key)

No encoding supplied: defaulting to UTF-8.



In [20]:
transfers

date,type,teams.in.id,teams.in.name,teams.in.logo,teams.out.id,teams.out.name,teams.out.logo,player.id,player.name
<date>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>
2099-09-01,Loan,35,Bournemouth,https://media.api-sports.io/football/teams/35.png,33,Manchester United,https://media.api-sports.io/football/teams/33.png,19609,J. O&apos;Shea
2098-08-01,€ 750K,33,Manchester United,https://media.api-sports.io/football/teams/33.png,1828,York,https://media.api-sports.io/football/teams/1828.png,115964,J. Greening
2094-07-01,Loan,59,Preston,https://media.api-sports.io/football/teams/59.png,33,Manchester United,https://media.api-sports.io/football/teams/33.png,113755,D. Beckham
2024-07-01,N/A,33,Manchester United,https://media.api-sports.io/football/teams/33.png,748,Burton Albion,https://media.api-sports.io/football/teams/748.png,282133,J. Hugill
2024-07-01,N/A,33,Manchester United,https://media.api-sports.io/football/teams/33.png,1351,Port Vale,https://media.api-sports.io/football/teams/1351.png,303010,D. Gore
2024-07-01,N/A,33,Manchester United,https://media.api-sports.io/football/teams/33.png,715,Granada CF,https://media.api-sports.io/football/teams/715.png,70078,F. Pellistri
2024-07-01,N/A,502,Fiorentina,https://media.api-sports.io/football/teams/502.png,33,Manchester United,https://media.api-sports.io/football/teams/33.png,74,S. Amrabat
2024-07-01,N/A,33,Manchester United,https://media.api-sports.io/football/teams/33.png,536,Sevilla,https://media.api-sports.io/football/teams/536.png,180560,H. Mejbri
2024-07-01,N/A,33,Manchester United,https://media.api-sports.io/football/teams/33.png,546,Getafe,https://media.api-sports.io/football/teams/546.png,897,M. Greenwood


The following package(s) will be updated in the lockfile:

# CRAN -----------------------------------------------------------------------
- MASS   [* -> 7.3-60.0.1]

- Lockfile written to "~/MDS Files/BLOCK 4/DATA 534/Project/DATA534_Project_Group8/Extras/renv.lock".
